In [1]:
import os

os.chdir(r'C:\Users\carlo\GitHub\ProjectBuffalo')

import buffalo.ingestion as ingestion
import buffalo.predictor as predictor
import buffalo.algorithm as algorithm
import buffalo.predictor.models as modeling
import torch
import pickle

In [2]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [3]:
ingestor.load_data(r'cached_data/ingestion.sqlite')

In [4]:
target_symbol = 'GE'

In [5]:
ingestor.data['ADJUSTED_DAILY_STOCK'].symbol.unique()

array(['AAPL', 'MSFT', 'IBM', 'META', 'JNJ', 'PFE', 'UNH', 'MARK', 'XLV',
       'JPM', 'BAC', 'GS', 'MS', 'XLF', 'PG', 'KO', 'PEP', 'NKE', 'XLP',
       'GE', 'HON', 'UTX', 'MMM', 'XLI'], dtype=object)

In [6]:
target_stock = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol == @target_symbol')[['open', 'high', 'low', 'adjusted_close', 'volume']]

In [7]:
other_stocks = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol != @target_symbol')

In [ ]:
other_stocks

In [8]:
for symbol in ['MSFT', 'IBM', 'JNJ', 'PFE', 'UNH', 'XLV', 'JPM', 'BAC', 'GS', 'XLF', 'AAPL', 'KO', 'PEP', 'NKE', 'XLP', 'PG', 'HON', 'MMM', 'XLI']:
    temp = other_stocks[other_stocks['symbol'] == symbol][['open', 'high', 'low', 'adjusted_close', 'volume']].rename(columns={'adjusted_close': 'close'})
    temp.columns = symbol + '_' + temp.columns
    target_stock = predictor.util.align_dataframe_by_time(target_stock, temp)

ValueError: Indexes have overlapping values: Index(['AAPL_open', 'AAPL_high', 'AAPL_low', 'AAPL_close', 'AAPL_volume'], dtype='object')

In [ ]:
target_income_statement = ingestor.data['COMPANY_INCOME_STATEMENT'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_balance_sheet = ingestor.data['COMPANY_BALANCE_SHEET'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_cash_flow = ingestor.data['COMPANY_CASH_FLOW'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function', 'net_income']).dropna(axis=1, how='all')
fed_funds_rate = ingestor.data['FEDERAL_FUNDS_RATE'][['value']].rename(columns={'value': 'effective_federal_funds_rate'}).dropna(axis=1, how='all')
payroll = ingestor.data['NONFARM_PAYROLL'][['value']].rename(columns={'value': 'total_nonfarm_payroll'}).dropna(axis=1, how='all')
cpi = ingestor.data['CPI'][['value']].rename(columns={'value': 'consumer_price_index'}).dropna(axis=1, how='all')
unemployment = ingestor.data['UNEMPLOYMENT'][['value']].rename(columns={'value': 'unemployment_rate'}).dropna(axis=1, how='all')
real_gdp = ingestor.data['REAL_GDP'][['value']].rename(columns={'value': 'real_gross_domestic_product'}).dropna(axis=1, how='all')
real_gdp_per_capita = ingestor.data['REAL_GDP_PER_CAPITA'][['value']].rename(columns={'value': 'real_gross_domestic_product_per_capita'})
treasury_yield = ingestor.data['TREASURY_YIELD'][['value', 'maturity']].pivot(columns=['maturity'], values=['value']).dropna(axis=1, how='all')
treasury_yield.columns = 'treasury_yield_' + treasury_yield.columns.droplevel(level=0)

In [ ]:
sma = ingestor.data['SMA'].query('symbol == @target_symbol & interval == "daily"')
roc = ingestor.data['ROC'].query('symbol == @target_symbol & interval == "daily"')
ht_sine = ingestor.data['HT_SINE'].query('symbol == @target_symbol & interval == "daily"')
mom = ingestor.data['MOM'].query('symbol == @target_symbol & interval == "daily"')
sma = sma.pivot(columns=['time_period', 'series_type'], values=['sma']).dropna(axis=1, how='all')
sma.columns = sma.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, sma)
roc = roc.pivot(columns=['time_period', 'series_type'], values=['roc']).dropna(axis=1, how='all')
roc.columns = roc.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, roc)
ht_sine = ht_sine.pivot(columns=['time_period', 'series_type'], values=['lead_sine', 'sine']).dropna(axis=1, how='all')
ht_sine.columns = ht_sine.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, ht_sine)
mom = mom.pivot(columns=['time_period', 'series_type'], values=['mom']).dropna(axis=1, how='all')
mom.columns = mom.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, mom)

In [ ]:
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, fed_funds_rate)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, payroll)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, cpi)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, unemployment)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp_per_capita)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, treasury_yield)
print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_income_statement)
#print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_balance_sheet)
#print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_cash_flow)
#print(target_stock.shape)

In [ ]:
pickle.dump(target_stock, open(r'cached_data/target_stock.pickle', 'wb'))

In [ ]:
target_stock = pickle.load(open('cached_data/target_stock.pickle', 'rb'))

In [ ]:
time_series_data = predictor.util.TimeSeriesData(endog=target_stock[['adjusted_close']], exog=target_stock.drop(columns=['adjusted_close']), seq_len=180, label_len=1, name=f'DAILY_ADJUSTED_CLOSE_{target_symbol}')

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=2, dropout=0.2, bidirectional=False, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(rnn, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=2, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(rnn, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=3, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(rnn, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=4, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(rnn, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=5, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(rnn, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
lstm = modeling.LSTM(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=2, dropout=0.2, bidirectional=False, use_gpu=True)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(lstm, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
lstm = modeling.LSTM(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=2, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(lstm, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
lstm = modeling.LSTM(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=3, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(lstm, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
lstm = modeling.LSTM(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=4, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(lstm, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
lstm = modeling.LSTM(input_size=target_stock.shape[1], hidden_size=64, output_size=1, num_layers=5, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001, weight_decay=0.001)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model(lstm, optimizer, loss_func, time_series_data, 30, 0.2, 5, clip_grad=1, batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record.plot_training_records()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=2, dropout=0.2, bidirectional=False, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=30, epochs_per_update=5, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction

In [ ]:
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')

In [ ]:
training_record.plot_training_records()


In [ ]:
training_record.plot_logs()

In [ ]:
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 1)

In [ ]:
training_record.plot_residuals()